In [1]:
import os
import pandas as pd


In [2]:
os.chdir("../")

In [3]:
%pwd

'e:\\machine_learning1121\\machine_learning120'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    base_score: float
    booster: str
    
    target_column: float

In [5]:
from src.mlProject3.constants import *
from src.mlProject3.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.XGBRegressor
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            base_score = params.base_score,
            booster = params.booster,
            target_column = schema.name
            
        )

        return model_trainer_config

In [7]:
import os
from src.mlProject3 import logger
from xgboost import XGBRegressor
import joblib


In [8]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]


        lr = XGBRegressor(alpha=self.config.base_score, booster=self.config.booster, random_state=42)
        lr.fit(train_x, train_y)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))



In [9]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-08-03 14:02:45,124: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-03 14:02:45,124: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-03 14:02:45,143: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-08-03 14:02:45,145: INFO: common: created directory at: artifacts]
[2024-08-03 14:02:45,146: INFO: common: created directory at: artifacts/model_trainer]
